## Preparation

In [1]:
!cp ../input/w2vu-cvsv-checkpoints-cpu10/checkpoint_best.pt .
!cp ../input/w2vu-cvsv-checkpoints-cpu10/checkpoint_last.pt .

In [2]:
%%capture
!conda install -c pykaldi pykaldi -y

In [3]:
%cd /tmp

/tmp


In [4]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...


remote: Enumerating objects: 28296, done.


remote: Total 28296 (delta 0), reused 0 (delta 0), pack-reused 28296
Receiving objects: 100% (28296/28296), 11.79 MiB | 6.01 MiB/s, done.


Resolving deltas: 100% (21294/21294), done.


In [5]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...


remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.


remote: Compressing objects: 100% (79/79), done.


remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 4.22 MiB/s, done.


Resolving deltas: 100% (7956/7956), done.


In [6]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [7]:

%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [8]:
%%capture
%cd /tmp/kenlm
!python setup.py install
%cd /tmp

In [9]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [10]:
%cd /tmp/fairseq/

/tmp/fairseq


In [11]:
%%capture
!python setup.py install

In [12]:
%cd /tmp/fairseq/

/tmp/fairseq


In [13]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [14]:
%%capture
!pip install editdistance

## GAN

In [15]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=true \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [16]:
!bash rungan.sh

[2021-06-06 21:28:05,147][HYDRA] Launching 5 jobs locally
[2021-06-06 21:28:05,147][HYDRA] 	#0 : task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=True checkpoint.save_dir=/kaggle/working common.seed=0


[2021-06-06 21:28:07,402][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tb', 'wandb_project': None, 'azureml_logging': False, 'seed': 0, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': True, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overri

[2021-06-06 21:28:07,537][fairseq_cli.train][INFO] - Wav2vec_U(
  (discriminator): Discriminator(
    (net): Sequential(
      (0): Conv1d(45, 384, kernel_size=(6,), stride=(1,), padding=(5,))
      (1): SamePad()
      (2): Dropout(p=0.0, inplace=False)
      (3): Sequential(
        (0): Conv1d(384, 384, kernel_size=(6,), stride=(1,), padding=(5,))
        (1): SamePad()
        (2): Dropout(p=0.0, inplace=False)
        (3): GELU()
      )
      (4): Conv1d(384, 1, kernel_size=(6,), stride=(1,), padding=(5,))
      (5): SamePad()
    )
  )
  (segmenter): JoinSegmenter()
  (generator): Generator(
    (dropout): Dropout(p=0.1, inplace=False)
    (proj): Sequential(
      (0): TransposeLast()
      (1): Conv1d(512, 45, kernel_size=(4,), stride=(1,), padding=(2,), bias=False)
      (2): TransposeLast()
    )
  )
)
[2021-06-06 21:28:07,537][fairseq_cli.train][INFO] - task: UnpairedAudioText
[2021-06-06 21:28:07,537][fairseq_cli.train][INFO] - model: Wav2vec_U
[2021-06-06 21:28:07,538][fa

[2021-06-06 21:28:07,624][fairseq.data.extracted_features_dataset][INFO] - loaded 2019, skipped 0 samples
[2021-06-06 21:28:07,624][fairseq.tasks.unpaired_audio_text][INFO] - split valid has unpaired text? False
[2021-06-06 21:28:07,633][fairseq_cli.train][INFO] - training on 1 devices (GPUs/TPUs)
[2021-06-06 21:28:07,634][fairseq_cli.train][INFO] - max tokens per device = None and max sentences per device = 160
[2021-06-06 21:28:07,635][fairseq.trainer][INFO] - Preparing to load checkpoint /kaggle/working/checkpoint_last.pt


[2021-06-06 21:28:07,994][fairseq.trainer][INFO] - Loaded checkpoint /kaggle/working/checkpoint_last.pt (epoch 4913 @ 78592 updates)
[2021-06-06 21:28:07,995][fairseq.trainer][INFO] - loading train data for epoch 4913


...

[2021-06-07 06:12:19,071][valid][INFO] - {"epoch": 5056, "valid_loss": "1.001", "valid_ntokens": "3039.79", "valid_nsentences": "144.214", "valid_lm_score_sum": "-89564.9", "valid_num_pred_chars": "48284", "valid_vocab_seen_pct": "0.898955", "valid_uer": "100.061", "valid_weighted_lm_ppl": "74.6506", "valid_lm_ppl": "60.3266", "valid_wps": "17151", "valid_wpb": "3039.8", "valid_bsz": "144.2", "valid_num_updates": "80896", "valid_best_weighted_lm_ppl": "73.0296"}
[2021-06-07 06:12:19,074][fairseq.checkpoint_utils][INFO] - Preparing to save checkpoint for epoch 5056 @ 80896 updates
[2021-06-07 06:12:19,076][fairseq.trainer][INFO] - Saving checkpoint to /kaggle/working/checkpoint_last.pt


[2021-06-07 06:12:19,127][fairseq.trainer][INFO] - Finished saving checkpoint to /kaggle/working/checkpoint_last.pt
[2021-06-07 06:12:19,127][fairseq.checkpoint_utils][INFO] - Saved checkpoint /kaggle/working/checkpoint_last.pt (epoch 5056 @ 80896 updates, score 74.65060222000588) (writing took 0.05264066300151171 seconds)
[2021-06-07 06:12:19,128][fairseq_cli.train][INFO] - end of epoch 5056 (average epoch stats below)
[2021-06-07 06:12:19,131][train][INFO] - {"epoch": 5056, "train_loss": "3.24", "train_ntokens": "145.688", "train_nsentences": "145.688", "train_temp": "0.1", "train_code_ppl": "8.795", "train_loss_code_pen": "0.332", "train_loss_smoothness": "2.095", "train_loss_dense_g": "4.156", "train_lm_score_sum": 0.0, "train_num_pred_chars": 0.0, "train_loss_grad_pen": "0.073", "train_loss_dense_d": "0.029", "train_loss_token_d": "0.031", "train_wps": "9.7", "train_ups": "0.07", "train_wpb": "145.7", "train_bsz": "145.7", "train_num_updates": "80896", "train_lr_discriminator": "0

[2021-06-07 06:12:19,191][fairseq.trainer][INFO] - begin training epoch 5057
[2021-06-07 06:12:19,192][fairseq_cli.train][INFO] - Start iterating over samples


[2021-06-07 06:13:21,433][train_inner][INFO] - {"epoch": 5057, "update": 5056.25, "loss": "3.363", "ntokens": "146.26", "nsentences": "146.26", "temp": "0.1", "code_ppl": "8.694", "loss_code_pen": "0.327", "loss_smoothness": "2.075", "loss_dense_g": "4.085", "lm_score_sum": 0.0, "num_pred_chars": 0.0, "loss_grad_pen": "0.109", "loss_dense_d": "0.028", "loss_token_d": "0.029", "wps": "9.9", "ups": "0.07", "wpb": "146.3", "bsz": "146.3", "num_updates": "80900", "lr_discriminator": "0.0005", "lr_generator": "0.0004", "gnorm": "34.345", "clip": "76", "train_wall": "1451", "wall": "31514"}


[2021-06-07 06:16:26,878][fairseq_cli.train][INFO] - begin validation on "valid" subset
